In [ ]:
import cv2
import mediapipe as mp
import time

class handDetector():
    def __init__(self,static_image_mode=False,max_num_hands=2,model_complexity=1,min_detection_confidence=0.5,min_tracking_confidence=0.5):
        self.static_image_mode=static_image_mode
        self.max_num_hands=max_num_hands
        self.model_complexity=model_complexity
        self.min_detection_confidence=min_detection_confidence
        self.min_tracking_confidence=min_tracking_confidence
        self.mpHands=mp.solutions.hands
        self.hands=self.mpHands.Hands(self.static_image_mode,self.max_num_hands,self.model_complexity,self.min_detection_confidence,self.min_tracking_confidence) #declaring object hands from mp.solutions.hands
        self.mpDraw=mp.solutions.drawing_utils
        self.tipIds=[4,8,12,16,20]   #tips of all the fingers
    def findHands(self,img,setDraw=True):
        imgRGB=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)  #converting because hands object only uses RGB images
        self.results=self.hands.process(imgRGB)
        if self.results.multi_hand_landmarks:
            for handLms in  self.results.multi_hand_landmarks:
                if setDraw:
                    self.mpDraw.draw_landmarks(img,handLms,self.mpHands.HAND_CONNECTIONS)   #for drawing connections
                
        return img
    def findPos(self,img,handNo=0,setDraw=True):
        self.lmList=[]
        if self.results.multi_hand_landmarks:
            myHand=self.results.multi_hand_landmarks[handNo]
            for id,lm in enumerate(myHand.landmark):
            #print(id,lm)
                h,w,c=img.shape
                cx,cy=int(lm.x*w),int(lm.y*h)
            #print(id,cx,cy)
                self.lmList.append([id,cx,cy])
                  ##if id==0:
                if setDraw:
                    cv2.circle(img,(cx,cy),15,(135,81,203),cv2.FILLED)
        return self.lmList
        #for handLms in  results.multi_hand_landmarks:
        
    def fingersUp(self):
        fingers=[]
        # Thumb
        if self.lmList[self.tipIds[0]][1]< self.lmList[self.tipIds[0] - 1][1]:
            fingers.append(1)
        else:
            fingers.append(0)

        # 4 Fingers
        for id in range(1, 5):
            if self.lmList[self.tipIds[id]][2] < self.lmList[self.tipIds[id] - 2][2]:
                fingers.append(1)
            else:
                fingers.append(0)
        return fingers
        
        
def main():
    pTime=0
    cTime=0
    cap=cv2.VideoCapture(0)
    detector=handDetector()

    while True:
        success,img=cap.read()
        img=detector.findHands(img)
        lmList=detector.findPos(img ,setDraw=False)
        if len(lmList)!=0:
               print(lmList[4])
                
        cTime=time.time()
        fps=1/(cTime-pTime)
        pTime=cTime
        cv2.putText(img,str(int(fps)),(10,70),cv2.FONT_HERSHEY_PLAIN,3,(135,81,20),3)        
        cv2.imshow("Image",img)
        cv2.waitKey(1)

if __name__== "__main__":
    main()